In [ ]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time

# 东京 Apple Store 门店列表（ID 和名称）
tokyo_stores = [
    {"id": "R079", "name": "Apple Marunouchi"},
    {"id": "R119", "name": "Apple Ginza"},
    {"id": "R128", "name": "Apple Shibuya"},
    {"id": "R224", "name": "Apple Omotesando"},
    {"id": "R718", "name": "Apple Omotesando"},
    {"id": "R710", "name": "Apple Shinjuku"}
]

# API 基础 URL
base_url = "https://www.apple.com/jp/shop/fulfillment-messages"
init_url = "https://www.apple.com/jp/"
# SKU 参数（可替换为 'MG864J/A' 如果需要）
sku = "MFY94J/A"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'ja-JP,ja;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Referer': 'https://www.apple.com/jp/shop/buy-iphone/iphone-17',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Connection': 'keep-alive',
    'Origin': 'https://www.apple.com'
}
# 创建 Session 并添加 retries
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[502, 503, 504, 541])
session.mount('https://', HTTPAdapter(max_retries=retries))

# 先访问初始化页面以设置 cookie
try:
    init_response = session.get(init_url, headers=headers, timeout=10)
    print(f"初始化页面状态码: {init_response.status_code}")
    print(f"设置的 Cookies: {session.cookies.get_dict()}")
except Exception as e:
    print(f"初始化失败: {str(e)}")

# 循环查询每个门店
for store in tokyo_stores:
    params = {
        "fae": "true",
        "pl": "true",
        "mts.0": "regular",
        "mts.1": "compact",
        "cppart": "UNLOCKED_JP",
        "parts.0": sku,
        "searchNearby": "true",
        "store": store["id"]
    }

    try:
        response = session.get(base_url, params=params, headers=headers, timeout=10)
        print(f"查询 {store['name']} 状态码: {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            stores = data.get('body', {}).get('content', {}).get('pickupMessage', {}).get('stores', [])
            if stores:
                target_store = next((s for s in stores if s.get('storeNumber') == store["id"]), stores[0])
                parts_availability = target_store.get('partsAvailability', {}).get(sku, {})
                availability = parts_availability.get('pickupDisplay', '未知')
                pickup_time = parts_availability.get('pickupSearchQuote', '未知')
                buyable = parts_availability.get('isBuyable', '未知')
            else:
                availability = '无门店数据'
                pickup_time = '未知'
                buyable = '未知'

            print(f"门店: {store['name']} (ID: {store['id']})")
            print(f"可用性: {availability}")
            print(f"取货时间估计: {pickup_time}")
            print(f"可购买: {buyable}")
            print("---")
        else:
            print(f"错误详情: {response.text[:200]}...")
    except Exception as e:
        print(f"查询 {store['name']} 异常: {str(e)}")

    time.sleep(2)  # 延迟避免限制

In [12]:
import requests

# 东京 Apple Store 门店列表（包括 R718 为 Marunouchi）
tokyo_stores = [
    {"id": "R718", "name": "Apple Marunouchi"},
    {"id": "R010", "name": "Apple Ginza"},
    {"id": "R095", "name": "Apple Shibuya"},
    {"id": "R120", "name": "Apple Omotesando"},
    {"id": "R124", "name": "Apple Shinjuku"}
]

# API 基础 URL
base_url = "https://www.apple.com/jp/shop/fulfillment-messages"

# SKU 参数
sku = "MFY94J/A"

# 循环查询每个门店
for store in tokyo_stores:
    params = {
        "fae": "true",
        "pl": "true",
        "mts.0": "regular",
        "mts.1": "compact",
        "cppart": "UNLOCKED_JP",
        "parts.0": sku,
        "searchNearby": "true"
    }

    response = requests.get(base_url, params=params)
    print("url:{}".format(response.url))
    print(f"查询 {store['name']} 失败，状态码: {response.status_code}")

    # if response.status_code == 200:
    #     data = response.json()
    #     # 解析响应：查找 stores 数组中的第一个门店（或指定门店）的可用性
    #     stores = data.get('body', {}).get('content', {}).get('pickupMessage', {}).get('stores', [])
    #     if stores:
    #         # 尝试找到匹配门店的可用性
    #         target_store = next((s for s in stores if s.get('storeNumber') == store["id"]), stores[0])
    #         parts_availability = target_store.get('partsAvailability', {}).get(sku, {})
    #         availability = parts_availability.get('pickupDisplay', '未知')
    #         pickup_time = parts_availability.get('pickupSearchQuote', '未知')
    #         buyable = parts_availability.get('isBuyable', '未知')
    #     else:
    #         availability = '无门店数据'
    #         pickup_time = '未知'
    #         buyable = '未知'
    #
    #     print(f"门店: {store['name']} (ID: {store['id']})")
    #     print(f"可用性: {availability}")
    #     print(f"取货时间估计: {pickup_time}")
    #     print(f"可购买: {buyable}")
    #     print("---")
    # else:
    #     print(f"查询 {store['name']} 失败，状态码: {response.status_code}")

url:https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MFY94J%2FA&searchNearby=true
查询 Apple Marunouchi 失败，状态码: 503
url:https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MFY94J%2FA&searchNearby=true
查询 Apple Ginza 失败，状态码: 503
url:https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MFY94J%2FA&searchNearby=true
查询 Apple Shibuya 失败，状态码: 503
url:https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MFY94J%2FA&searchNearby=true
查询 Apple Omotesando 失败，状态码: 503
url:https://www.apple.com/jp/shop/fulfillment-messages?fae=true&pl=true&mts.0=regular&mts.1=compact&cppart=UNLOCKED_JP&parts.0=MFY94J%2FA&searchNearby=true
查询 Apple Shinjuku 失败，状态码: 503


In [4]:
from typing import Protocol, Dict, Callable, Optional,List
import os
from functools import lru_cache
from pathlib import Path
import re
import pandas as pd
from typing import Optional, Tuple
from urllib.parse import urlparse

from typing import Dict, Optional, List
import re
import pandas as pd
from pathlib import Path
from datetime import datetime
from dateutil import parser as dateparser
import pytz
TOKYO = pytz.timezone("Asia/Tokyo")





In [5]:
df_1 = pd.read_csv("/Users/syu/PycharmProjects/YamagotiProjects/shop1.csv")
df_1

,JAN,price,time-scraped
0,4549995649291,215000,2025-10-07 17:56:10.601419+09:00
1,4549995649307,205000,2025-10-07 17:56:10.601689+09:00
2,4549995649284,205000,2025-10-07 17:56:10.601921+09:00
3,4549995649321,243000,2025-10-07 17:56:10.602135+09:00
4,4549995649338,235000,2025-10-07 17:56:10.602345+09:00
5,4549995649314,233000,2025-10-07 17:56:10.602557+09:00
6,4549995649352,281000,2025-10-07 17:56:10.602766+09:00
7,4549995649369,278000,2025-10-07 17:56:10.602974+09:00
8,4549995649345,278000,2025-10-07 17:56:10.603195+09:00
9,4549995649383,331000,2025-10-07 17:56:10.603406+09:00


In [6]:
res = clean_shop1(df_1)
res

NameError: name '__file__' is not defined